In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Ссылка на главную страницу книжного магазина для скрапинга

In [33]:
url = 'http://books.toscrape.com/'

# Функция для извлечения данных о книгах

In [34]:
def scrape_books(url):
    # Пустые списки для хранения данных о книгах
    names = []
    prices = []
    availability = []
    descriptions = []

    # Запрос к странице
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Находим все ссылки на книги
    book_links = soup.find_all('h3')

    # Цикл по каждой книге
    for link in book_links:
        book_url = link.find('a')['href']
        book_response = requests.get(url + book_url)
        book_soup = BeautifulSoup(book_response.content, 'html.parser')

        # Извлекаем данные о книге
        name = book_soup.find('h1').text
        price = float(book_soup.find('p', class_='price_color').text.split('£')[1])
        stock = int(re.search(r'\d+', book_soup.find('p', class_='instock availability').text).group())
        description = book_soup.find('meta', attrs={'name': 'description'})['content']

        # Добавляем данные в соответствующие списки
        names.append(name)
        prices.append(price)
        availability.append(stock)
        descriptions.append(description)

    return names, prices, availability, descriptions


# Извлекаем данные

In [35]:
names, prices, availability, descriptions = scrape_books(url)

# Создаем DataFrame

In [36]:
df = pd.DataFrame({
    'Name': names,
    'Price': prices,
    'Availability': availability,
    'Description': descriptions
})

# Выводим DataFrame

In [37]:
df

,Name,Price,Availability,Description
0,A Light in the Attic,51.77,22,\n It's hard to imagine a world without A L...
1,Tipping the Velvet,53.74,20,"\n ""Erotic and absorbing...Written with sta..."
2,Soumission,50.10,20,\n Dans une France assez proche de la nôtre...
3,Sharp Objects,47.82,20,"\n WICKED above her hipbone, GIRL across he..."
4,Sapiens: A Brief History of Humankind,54.23,20,\n From a renowned historian comes a ground...
5,The Requiem Red,22.65,19,\n Patient Twenty-nine.A monster roams the ...
6,The Dirty Little Secrets of Getting Your Dream...,33.34,19,\n Drawing on his extensive experience eval...
7,The Coming Woman: A Novel Based on the Life of...,17.93,19,"\n ""If you have a heart, if you have a soul..."
8,The Boys in the Boat: Nine Americans and Their...,22.60,19,\n For readers of Laura Hillenbrand's Seabi...
9,The Black Maria,52.15,19,"\n Praise for Aracelis Girmay:""[Girmay's] e..."


# Сохраняем DataFrame в JSON файл

In [38]:
df.to_json('books_data.json', orient='records')